In [ ]:
#!/usr/bin/env python
# IBKR Market Order Script for Option Spreads

import sqlite3
import pandas as pd
import datetime
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order
from ibapi.utils import iswrapper
import time
import threading
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class IBWrapper(EWrapper):
    def __init__(self):
        super().__init__()
        self.next_order_id = None
        self.contract_details = {}
        self.mid_prices = {}

    @iswrapper
    def nextValidId(self, orderId: int):
        self.next_order_id = orderId
        logger.info(f"Next Valid Order ID: {orderId}")
    
    @iswrapper
    def tickPrice(self, reqId, tickType, price, attrib):
        # We're only interested in bid (1) and ask (2) prices
        if tickType == 1:  # Bid
            if reqId not in self.mid_prices:
                self.mid_prices[reqId] = {"bid": None, "ask": None}
            self.mid_prices[reqId]["bid"] = price
            logger.info(f"Received bid price for req_id {reqId}: {price}")
        elif tickType == 2:  # Ask
            if reqId not in self.mid_prices:
                self.mid_prices[reqId] = {"bid": None, "ask": None}
            self.mid_prices[reqId]["ask"] = price
            logger.info(f"Received ask price for req_id {reqId}: {price}")
    
    @iswrapper
    def tickOptionComputation(self, reqId, tickType, tickAttrib, impliedVol, delta, optPrice, pvDividend, gamma, vega, theta, undPrice):
        # If we don't have market data, try to use the option computation data
        # tickType 12 = last price, tickType 13 = model price
        if optPrice is not None and (tickType == 12 or tickType == 13):
            if reqId not in self.mid_prices:
                self.mid_prices[reqId] = {"bid": None, "ask": None, "last": None, "model": None}
            
            if tickType == 12:  # Last price
                self.mid_prices[reqId]["last"] = optPrice
                logger.info(f"Received last price for req_id {reqId}: {optPrice}")
            elif tickType == 13:  # Model price
                self.mid_prices[reqId]["model"] = optPrice
                logger.info(f"Received model price for req_id {reqId}: {optPrice}")

    @iswrapper
    def error(self, reqId, errorCode, errorString, advancedOrderRejectJson="", connectionClosed=False):
        logger.error(f"Error {errorCode}: {errorString}")
        
        # Check if connection is closed
        if connectionClosed:
            logger.warning("Connection to IBKR was closed")

class IBClient(EClient):
    def __init__(self, wrapper):
        super().__init__(wrapper)

class IBApp(IBWrapper, IBClient):
    def __init__(self):
        IBWrapper.__init__(self)
        IBClient.__init__(self, wrapper=self)
        self.connected = False
        self.order_placed = False
        self.request_ids = {}

    def create_option_contract(self, symbol, expiry, strike, right):
        contract = Contract()
        contract.symbol = symbol
        contract.secType = "OPT"
        contract.exchange = "SMART"
        contract.currency = "USD"
        
        # Format the expiry date properly for IBKR (remove hyphens)
        # Convert from "2025-03-28" to "20250328" format
        expiry_formatted = expiry.replace("-", "")
        
        contract.lastTradeDateOrContractMonth = expiry_formatted
        contract.strike = strike
        contract.right = right
        contract.multiplier = "100"
        
        logger.info(f"Created option contract: {symbol} {expiry_formatted} {strike} {right}")
        return contract
    
    def create_market_order(self, action, quantity):
        order = Order()
        order.action = action
        order.orderType = "MKT"
        order.totalQuantity = quantity
        return order

    def get_price_data(self, contract, req_id):
        """Request market data with increased timeout and better logging."""
        logger.info(f"Requesting market data for {contract.symbol} {contract.strike} {contract.right} (req_id: {req_id})")
        self.reqMktData(req_id, contract, "", False, False, [])
        
        # Allow more time for market data to arrive
        wait_time = 5  # Increased from 2 to 5 seconds
        time.sleep(wait_time)
        
        # Log the received data (or lack thereof)
        if req_id in self.mid_prices:
            bid = self.mid_prices[req_id].get("bid")
            ask = self.mid_prices[req_id].get("ask")
            if bid is not None and ask is not None:
                logger.info(f"Received market data for req_id {req_id}: Bid={bid}, Ask={ask}")
            else:
                logger.warning(f"Incomplete market data for req_id {req_id}: Bid={bid}, Ask={ask}")
        else:
            logger.warning(f"No market data received for req_id {req_id}")

def get_strategies_for_date(date_str):
    """Get option strategies from database for a specific date."""
    conn = sqlite3.connect('database/option_strategies.db')
    
    # Convert date_str to datetime for filtering
    target_date = pd.to_datetime(date_str)
    
    # Format for ISO timestamp comparison (handles timestamps like '2025-03-10T08:41:01.483574')
    start_date = target_date.strftime('%Y-%m-%d')
    
    # Query to get rows where scrape_date matches the target day
    # Using date() SQLite function to extract just the date part from the timestamp
    query = f"""
    SELECT * FROM option_strategies 
    WHERE date(scrape_date) = '{start_date}'
    AND timestamp_of_trigger IS NOT NULL
    """
    
    df = pd.read_sql_query(query, conn)
    conn.close()
    
    return df

def update_strategy_status(row_id, status, premium):
    """Update the strategy status and premium received in the database."""
    conn = sqlite3.connect('database/option_strategies.db')
    cursor = conn.cursor()
    
    # Premium is already in per-contract basis (multiplied by 100)
    cursor.execute(
        """
        UPDATE option_strategies 
        SET strategy_status = ?, premium_received = ?
        WHERE id = ?
        """, 
        (status, premium, row_id)
    )
    
    conn.commit()
    conn.close()

def run_trading_app(target_date=None):
    """
    Main function to process and place orders.
    
    Args:
        target_date (str, optional): Date string in 'YYYY-MM-DD' format.
            If None, today's date will be used.
    """
    # Use the provided date or default to today's date
    if target_date is None:
        target_date = datetime.datetime.now().strftime('%Y-%m-%d')
    
    logger.info(f"Processing strategies for date: {target_date}")
    
    # Get strategies for the target date
    df = get_strategies_for_date(target_date)
    
    if df.empty:
        logger.info(f"No strategies found for {target_date}")
        return
    
    logger.info(f"Found {len(df)} strategies to process")
    
    # Initialize IBKR app
    app = IBApp()
    
    # Connect to IBKR
    app.connect('127.0.0.1', 7497, 0)  # Default paper trading port
    
    # Start a thread to process IBKR messages
    ibkr_thread = threading.Thread(target=app.run)
    ibkr_thread.start()
    
    # Wait for connection and valid order ID
    timeout = 10
    start_time = time.time()
    while not app.next_order_id and time.time() - start_time < timeout:
        time.sleep(0.1)
    
    if not app.next_order_id:
        logger.error("Failed to connect to IBKR or get valid order ID")
        app.disconnect()
        return
    
    # Process each strategy
    for idx, row in df.iterrows():
        ticker = row['ticker']
        expiry = row['options_expiry_date']
        strategy_type = row['strategy_type']
        strike_buy = row['strike_buy']
        strike_sell = row['strike_sell']
        estimated_premium = row['estimated_premium']
        
        logger.info(f"Processing {strategy_type} for {ticker}, expiry {expiry}")
        
        # Determine contract details based on strategy type
        if strategy_type == 'Bear Call':
            # For Bear Call: Sell lower strike call, buy higher strike call
            sell_contract = app.create_option_contract(ticker, expiry, strike_sell, "C")
            buy_contract = app.create_option_contract(ticker, expiry, strike_buy, "C")
        elif strategy_type == 'Bull Put':
            # For Bull Put: Sell higher strike put, buy lower strike put
            sell_contract = app.create_option_contract(ticker, expiry, strike_sell, "P")
            buy_contract = app.create_option_contract(ticker, expiry, strike_buy, "P")
        else:
            logger.error(f"Unknown strategy type: {strategy_type}")
            continue
        
        # Get market data for both legs
        req_id_sell = app.next_order_id
        app.next_order_id += 1
        app.get_price_data(sell_contract, req_id_sell)
        
        req_id_buy = app.next_order_id
        app.next_order_id += 1
        app.get_price_data(buy_contract, req_id_buy)
        
        # Calculate mid prices for both legs
        try:
            # Check if we have the necessary market data
            if req_id_sell not in app.mid_prices:
                logger.error(f"No market data received for sell leg (req_id: {req_id_sell})")
                update_strategy_status(row['id'], 'missing market data', 0)
                continue
                
            if req_id_buy not in app.mid_prices:
                logger.error(f"No market data received for buy leg (req_id: {req_id_buy})")
                update_strategy_status(row['id'], 'missing market data', 0)
                continue
                
            # Try to get prices in this order: bid/ask first, then model, then last
            sell_price = None
            buy_price = None
            
            # For sell leg
            sell_bid = app.mid_prices[req_id_sell].get('bid')
            sell_ask = app.mid_prices[req_id_sell].get('ask')
            sell_model = app.mid_prices[req_id_sell].get('model')
            sell_last = app.mid_prices[req_id_sell].get('last')
            
            # For buy leg
            buy_bid = app.mid_prices[req_id_buy].get('bid')
            buy_ask = app.mid_prices[req_id_buy].get('ask')
            buy_model = app.mid_prices[req_id_buy].get('model')
            buy_last = app.mid_prices[req_id_buy].get('last')
            
            # Determine sell price (prefer bid/ask mid, then model, then last)
            if sell_bid is not None and sell_ask is not None:
                sell_price = (sell_bid + sell_ask) / 2
                logger.info(f"Using bid/ask mid for sell leg: {sell_price}")
            elif sell_model is not None:
                sell_price = sell_model
                logger.info(f"Using model price for sell leg: {sell_price}")
            elif sell_last is not None:
                sell_price = sell_last
                logger.info(f"Using last price for sell leg: {sell_price}")
            else:
                logger.error("No valid price data for sell leg")
                update_strategy_status(row['id'], 'no valid price data', 0)
                continue
                
            # Determine buy price (prefer bid/ask mid, then model, then last)
            if buy_bid is not None and buy_ask is not None:
                buy_price = (buy_bid + buy_ask) / 2
                logger.info(f"Using bid/ask mid for buy leg: {buy_price}")
            elif buy_model is not None:
                buy_price = buy_model
                logger.info(f"Using model price for buy leg: {buy_price}")
            elif buy_last is not None:
                buy_price = buy_last
                logger.info(f"Using last price for buy leg: {buy_price}")
            else:
                logger.error("No valid price data for buy leg")
                update_strategy_status(row['id'], 'no valid price data', 0)
                continue
            
            # Premium collected is the difference (sell price - buy price)
            premium_collected = sell_price - buy_price
            
            # Convert premium_collected to dollar value per contract (multiply by 100)
            premium_collected_dollar = premium_collected * 100
            
            logger.info(f"Prices - Sell: {sell_price}, Buy: {buy_price}")
            logger.info(f"Premium collected: {premium_collected} per share, ${premium_collected_dollar:.2f} per contract")
            logger.info(f"Estimated premium in database: ${estimated_premium:.2f} per contract")
            
            # Check if premium is sufficient - compare with estimated premium
            if premium_collected_dollar >= estimated_premium:
                # Place the order (vertical spread)
                order_id = app.next_order_id
                app.next_order_id += 1
                
                # Create and place order for sell leg
                sell_order = app.create_market_order("SELL", 1)
                app.placeOrder(order_id, sell_contract, sell_order)
                logger.info(f"Placed SELL order {order_id} for {ticker} {strike_sell}")
                
                # Create and place order for buy leg
                order_id = app.next_order_id
                app.next_order_id += 1
                buy_order = app.create_market_order("BUY", 1)
                app.placeOrder(order_id, buy_contract, buy_order)
                logger.info(f"Placed BUY order {order_id} for {ticker} {strike_buy}")
                
                # Update database - store the per-contract dollar amount (premium * 100)
                update_strategy_status(row['id'], 'order placed', premium_collected_dollar)
            else:
                # Premium too low, update database with the per-contract dollar amount
                update_strategy_status(row['id'], 'premium too low', premium_collected_dollar)
        
        except Exception as e:
            logger.error(f"Error processing order: {str(e)}")
            # Log more details about the exception for debugging
            import traceback
            logger.error(f"Exception details: {traceback.format_exc()}")
            update_strategy_status(row['id'], 'error', 0)
    
    # Clean up
    time.sleep(3)  # Give time for orders to process
    app.disconnect()
    logger.info("Disconnected from IBKR")

if __name__ == "__main__":
    # Run with today's date by default
    run_trading_app('2025-03-12')

2025-03-13 16:48:53,162 - __main__ - INFO - Processing strategies for date: 2025-03-12
2025-03-13 16:48:53,165 - __main__ - INFO - Found 1 strategies to process
2025-03-13 16:48:53,171 - ibapi.client - INFO - sent startApi
2025-03-13 16:48:53,172 - ibapi.utils - INFO - REQUEST startApi {}
2025-03-13 16:48:53,173 - ibapi.client - INFO - SENDING startApi b'\x00\x00\x00\x0871\x002\x000\x00\x00'
2025-03-13 16:48:53,173 - ibapi.utils - INFO - ANSWER connectAck {}
2025-03-13 16:48:53,179 - __main__ - INFO - Next Valid Order ID: 205
2025-03-13 16:48:53,218 - ibapi.utils - INFO - ANSWER managedAccounts {'accountsList': 'DU9233079'}
2025-03-13 16:48:53,219 - __main__ - ERROR - Error 1741884533178: 2104
2025-03-13 16:48:53,220 - __main__ - ERROR - Error 1741884533178: 2104
2025-03-13 16:48:53,220 - __main__ - ERROR - Error 1741884533179: 2104
2025-03-13 16:48:53,221 - __main__ - ERROR - Error 1741884533180: 2104
2025-03-13 16:48:53,221 - __main__ - ERROR - Error 1741884533180: 2104
2025-03-13 16

In [18]:
conn = sqlite3.connect('./database/option_strategies.db')

# Convert date_str to datetime for filtering
target_date = pd.to_datetime('2025-03-11')
start_date = target_date.strftime('%Y-%m-%d 00:00:00')
end_date = target_date.strftime('%Y-%m-%d 23:59:59')

# Query to get rows where scrape_date is within the target day
query = f"""
SELECT * FROM option_strategies 
WHERE scrape_date BETWEEN '{start_date}' AND '{end_date}'
AND timestamp_of_trigger IS NOT NULL
"""

df = pd.read_sql_query(query, conn)
conn.close()

In [16]:
df

,id,scrape_date,strategy_type,tab_name,ticker,trigger_price,strike_price,strike_buy,strike_sell,estimated_premium,item_id,options_expiry_date,date_info,timestamp_of_trigger,strategy_status,price_when_triggered,premium_received


In [19]:
start_date

'2025-03-11 00:00:00'

In [28]:
conn = sqlite3.connect('database/option_strategies.db')

# Convert date_str to datetime for filtering
target_date = pd.to_datetime('2025-03-11')
start_date = target_date.strftime('%Y-%m-%d 00:00:00')
end_date = target_date.strftime('%Y-%m-%d 23:59:59')

# Query to get rows where scrape_date is within the target day
query = f"""
SELECT * FROM option_strategies 
WHERE scrape_date BETWEEN '{start_date}' AND '{end_date}'
"""

df = pd.read_sql_query(query, conn)

conn.close()
df

,id,scrape_date,strategy_type,tab_name,ticker,trigger_price,strike_price,strike_buy,strike_sell,estimated_premium,item_id,options_expiry_date,date_info,timestamp_of_trigger,strategy_status,price_when_triggered,premium_received


In [29]:
start_date

'2025-03-11 00:00:00'

In [ ]:
2025-03-10T08:41:01.483574